In [1]:
import osmnx as ox
import networkx as nx
import random
import numpy as np

In [2]:
class MapGraph():
    def __init__(self, graph):
        self.graph = graph

    def distance_between(self, n1, n2):
        if 'length' in self.graph[n1][n2]:
            return self.graph[n1][n2]['length']
        else:
            return 99999999 # return a large number if 'length' attribute is not present

    def create_path(self, path_list):
        path = []
        for i in range(len(path_list)-1):
            path += nx.shortest_path(self.graph, path_list[i], path_list[i+1], weight='length')
        return path

    def evaluate_fitness(self, path):
        # Calculate the total distance of the path
        total_distance = sum([self.distance_between(path[i], path[i+1]) for i in range(len(path)-1)])
        return 1 / total_distance # Return the fitness as the reciprocal of the total distance

    def generate_initial_population(self, population_size):
        # Generate a random permutation of nodes as the initial population
        nodes = list(self.graph.nodes())
        initial_population = [random.sample(nodes, len(nodes)) for i in range(population_size)]
        return initial_population

    def select_parents(self, population, fitness_scores):
        # Select two parents from the population using tournament selection
        k = 5
        idx = np.random.choice(len(population), size=k, replace=False)
        tournament_population = [population[i] for i in idx]
        tournament_fitness_scores = [fitness_scores[i] for i in idx]
        idx1 = np.argmax(tournament_fitness_scores)
        tournament_population.pop(idx1)
        tournament_fitness_scores.pop(idx1)
        idx2 = np.argmax(tournament_fitness_scores)
        parent1 = tournament_population[idx1]
        parent2 = tournament_population[idx2]
        return parent1, parent2

    def crossover(self, parent1, parent2):
        # Perform crossover by selecting a random subsequence from parent1 and filling the remaining nodes with the nodes from parent2
        crossover_point = random.randint(0, len(parent1)-1)
        child = parent1[crossover_point:]
        for node in parent2:
            if node not in child:
                child.append(node)
        return child

    def mutate(self, individual):
        # Perform mutation by swapping two randomly selected nodes
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
        return individual

    def evolve_population(self, population, fitness_scores, elitism=True):
        # Perform elitism by selecting the best individual from the previous generation
        if elitism:
            elite_idx = np.argmax(fitness_scores)
            new_population = [population[elite_idx]]
        else:
            new_population = []

        # Generate new offspring using crossover and mutation
        while len(new_population) < len(population):
            parent1, parent2 = self.select_parents(population, fitness_scores)
            child = self.crossover(parent1, parent2)
            child